In [1]:
##### 8. 데이터 준비하기: 조인, 병합, 변형 #####

In [2]:
##### 8.1 계층적 색인 #####

In [3]:
# 계층적 색인은 축에 대해 다중 색인 단계를 지정할 수 있도록 해준다.
# 피벗테이블 생성 같은 그룹 기반의 작업을 할 때 중요하다.
# 예를 들어, 리스트의 리스트를 색이능로 하는 Series를 하나 생성하자.

import pandas as pd
import numpy as np

In [4]:
data = pd.Series(np.random.randn(9),
                index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                      [1, 2, 3, 1, 3, 1, 2, 2, 3]])

In [5]:
data

a  1    0.263430
   2   -1.623748
   3    1.215518
b  1    2.575256
   3    1.071513
c  1   -0.333965
   2   -0.284971
d  2   -1.874445
   3    2.800309
dtype: float64

In [6]:
# 지금 생성한 객체가 MultiIndex를 색인으로 하는 Series인데, 색인의 계층을 보여주고 있다.
# 바로 위 단계의 색인을 이용해 하위 계층을 직접 접근할 수 있다.

data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [7]:
# 계층적으로 색인된 객체는 데이터의 부분집합을 부분적 색인으로 접근하는 것이 가능하다.

data['b']

1    2.575256
3    1.071513
dtype: float64

In [8]:
data['b':'c']

b  1    2.575256
   3    1.071513
c  1   -0.333965
   2   -0.284971
dtype: float64

In [9]:
data.loc[['b', 'd']]

b  1    2.575256
   3    1.071513
d  2   -1.874445
   3    2.800309
dtype: float64

In [11]:
# 하위 계층의 객체를 선택하는 것도 가능하다.

data.loc[:, 2]

a   -1.623748
c   -0.284971
d   -1.874445
dtype: float64

In [14]:
# 데이터 재배열

data.unstack()

,1,2,3
a,0.263430,-1.623748,1.215518
b,2.575256,NaN,1.071513
c,-0.333965,-0.284971,NaN
d,NaN,-1.874445,2.800309


In [15]:
# unstack의 반대 작업으로 stack 메서드가 있다.

data.unstack().stack()

a  1    0.263430
   2   -1.623748
   3    1.215518
b  1    2.575256
   3    1.071513
c  1   -0.333965
   2   -0.284971
d  2   -1.874445
   3    2.800309
dtype: float64

In [18]:
# DataFrame에서는 두 축 모두 계층적 색인을 가질 수 있다.

frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                    index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                    columns=[['Ohio', 'Ohio', 'Colorado'],
                            ['Green', 'Red', 'Green']])

In [19]:
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [20]:
frame.index.names = ['key1', 'key2']

In [21]:
frame.columns.names = ['state', 'color']

In [22]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

In [23]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

In [25]:
MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                      names=['state',' color'])

AttributeError: 'DataFrame' object has no attribute 'from_arrays'

In [26]:
##### 8.1.1 계층의 순서를 바꾸고 정렬하기 #####

In [28]:
# 계층의 순서를 바꾸거나 데이터를 정렬해야하는 경우
# swaplevel : 넘겨받은 두 개의 계층번호나 이름이 뒤바뀐 새로운 객체를 반환

frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [29]:
# sort_index : 단일 계층에 속한 데이터 정렬

frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [30]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

In [31]:
##### 8.1.2 계층별 요약 통계 #####

In [32]:
# DataFrame과 Series의 기술통계와 요약 통계는 level 옵션을 가지고 있다.
# 어떤 한 축에 대해 합을 구하고 싶은 단계를 지정할 수 있는 옵션이다.

frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [33]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [35]:
##### 8.1.3 DataFrmae의 컬럼 사용하기 #####

In [36]:
# DataFrame에서 로우를 선택하기 위한 색인으로 하나 이상의 컬럼을 사용하는 경우가 많다.
# 로우의 색인을 DataFrame의 컬럼으로 옮겨보자.

frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                     'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0, 1, 2, 0 , 1, 2, 3]})

In [37]:
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


In [40]:
# set_index 함수는 하나 이상의 컬럼을 색인으로 하는 새로운 DataFrame을 생성한다.

frame2 = frame.set_index(['c', 'd'])

In [41]:
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [43]:
# 컬럼을 명시적으로 남겨두어야 DataFrame에서 삭제되지않는다.

frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [44]:
# reset_index 함수는 set_index와 반대되는 개념인데 계층적 색인 단계가 컬럼으로 이동한다.

frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1
